In [ ]:
from pathlib import Path
import pandas as pd
import xarray as xr
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from shapely.geometry import Point, Polygon, MultiPolygon
import shapely.plotting
import geojson
from dask.diagnostics import ProgressBar

import cartopy.crs as ccrs
import cartopy.feature as cfeature


pbar = ProgressBar()
pbar.register()

In [ ]:
start_date = "2022-02-01"
end_date = "2024-04-08"
list_date = pd.date_range(start=start_date, end=end_date, freq="D")
list_date = list_date.strftime("%Y-%m-%d")
cases_forecast = {
    "J+0": ["00H12H", "13H24H"],
    "J+1": ["25H36H", "37H48H"],
    "J+2": ["49H60H", "61H72H"],
    "J+3": ["73H84H", "85H96H"],
}
cases = [
    "00H12H",
    "13H24H",
]  # "25H36H", "37H48H", "49H60H", "61H72H", "73H84H", "85H96H", "97H102H"]
filename_to_save_template = (
    "/shared/home/antoine-2etavant/data/arpege/{date}_SP1_{case}.grib2"
)


filename = filename_to_save_template.format(date=list_date[-1], case=cases[1])

number_of_files_to_process = 3
filenames_morning = [
    filename_to_save_template.format(date=date, case=cases[0])
    for date in list_date[:number_of_files_to_process]
]
filenames_afternoon = [
    filename_to_save_template.format(date=date, case=cases[1])
    for date in list_date[:number_of_files_to_process]
]

filenames_morning_exist = [
    filename for filename in filenames_morning if Path(filename).exists()
]
filenames_afternoon_exist = [
    filename for filename in filenames_afternoon if Path(filename).exists()
]

filenames_morning = filenames_morning_exist
filenames_afternoon = filenames_afternoon_exist

try:
    ## remove a file that is not working properly
    filenames_afternoon.remove(
        "/shared/home/antoine-2etavant/data/arpege/2023-08-20_SP1_13H24H.grib2"
    )
except:
    pass

save_parquet = number_of_files_to_process == -1

# Open a list a Grib files

The following section opens a list of grib files and merge them intoo one XArray.

It requieres to av install Dask and CFGrid in addition to xarray, that can be install with
```bash
mamba install -y -c conda-forge dask xarray cfgrib
```

In [ ]:
KEYS_FILTER_SSPD = {
    "typeOfLevel": "surface",
    "cfVarName": "ssrd",
}
KEYS_FILTER_WIND = {
    "typeOfLevel": "heightAboveGround",
    "level": 10,
    "cfVarName": "si10",
}
KEYS_FILTER_T2M = {
    "typeOfLevel": "heightAboveGround",
    "level": 2,
    "cfVarName": "t2m",
}

In [ ]:
# create mask if point inside metropole
metropole_geojson_file = "./notebooks/datascience/metropole.geojson"
metropole = geojson.load(open(metropole_geojson_file))
polys_france = [Polygon(geo[0]) for geo in metropole["geometry"]["coordinates"]]
areas = [p.area for p in polys_france]

min_area = 0.2  # threshold to remove small islands
polys_france = [p for p, area in zip(polys_france, areas) if area > min_area]
print(f"number of polygons: {len(polys_france)}")
poly_france = MultiPolygon(polys_france)
# poly_france = polys_france[0]  # france metropolitan

bouns = poly_france.envelope.bounds
min_lon = bouns[0]
max_lon = bouns[2]
min_lat = bouns[1]
max_lat = bouns[3]

for poly in poly_france.geoms:
    shapely.plotting.plot_polygon(poly)

## The Solar Flux

This section reads the solar flux from the GRIB2 files, an uses XArray to process the means.

In [ ]:
def drop_no_step(ds):
    # Check if 'step' is in the dataset's coordinates
    # Used to find files that are not working properly
    if "step" not in ds.coords:
        display(ds)
    return ds


da_ssrd = xr.open_mfdataset(
    filenames_afternoon,
    engine="cfgrib",
    parallel=True,
    backend_kwargs={"filter_by_keys": KEYS_FILTER_SSPD},
    concat_dim="time",
    combine="nested",
    preprocess=drop_no_step,
).ssrd

In [ ]:
ssrd_france_large = da_ssrd.sel(step=np.timedelta64(1, "D")).where(
    (da_ssrd.longitude > min_lon)
    & (da_ssrd.longitude < max_lon)
    & (da_ssrd.latitude > min_lat)
    & (da_ssrd.latitude < max_lat),
    drop=True,
)
ssrd_france_large

In [ ]:
total_sun_flux_large = ssrd_france_large.sum(dim=["latitude", "longitude"])
total_sun_flux_large

In [ ]:
df_total_sun_flux_large = total_sun_flux_large.to_dataframe()[["ssrd"]]
df_total_sun_flux_large

In [ ]:
if save_parquet:
    df_total_sun_flux_large.to_parquet("sun_flux.parquet")

In [ ]:
df_total_sun_flux_large.plot()

## Processing the mean wind speed

In [ ]:
da_wind_10m_morning = xr.open_mfdataset(
    filenames_morning,
    engine="cfgrib",
    parallel=True,
    backend_kwargs={"filter_by_keys": KEYS_FILTER_WIND},
    concat_dim="time",
    combine="nested",
).si10

da_wind_10m_afternoon = xr.open_mfdataset(
    filenames_afternoon,
    engine="cfgrib",
    parallel=True,
    backend_kwargs={"filter_by_keys": KEYS_FILTER_WIND},
    concat_dim="time",
    combine="nested",
).si10
da_wind_10m = xr.concat(
    [
        da_wind_10m_morning,
        da_wind_10m_afternoon.where(
            da_wind_10m_afternoon.step < np.timedelta64(1, "D"), drop=True
        ),
    ],
    dim="step",
)
da_wind_10m

In [ ]:
wind_10m_large = da_wind_10m.where(
    (da_wind_10m.longitude > min_lon)
    & (da_wind_10m.longitude < max_lon)
    & (da_wind_10m.latitude > min_lat)
    & (da_wind_10m.latitude < max_lat),
    drop=True,
)
wind_10m_large

In [ ]:
total_wind_10m_large = (
    wind_10m_large.sum(dim=["latitude", "longitude"])
    .stack(time_step=("time", "step"))
    .set_index(time_step="valid_time")
)
total_wind_10m_large

In [ ]:
df_total_wind_10m_large = total_wind_10m_large.to_dataframe()[["si10"]]

In [ ]:
if save_parquet:
    df_total_wind_10m_large.to_parquet("total_wind_10m.parquet")
df_total_wind_10m_large.plot()

# Aggregating values in the Territory

The Above analysis only looked at the large average / total variable.

The idea of the two next sections is to select only the data inside a given territory

In [ ]:
# Define a function that checks if a point is inside the Polygon
def is_inside(lon, lat, polygons: MultiPolygon):
    return polygons.contains(Point(lon, lat))


def compute_mask_da(da: xr.DataArray, polygons: MultiPolygon):
    mask = xr.apply_ufunc(
        is_inside,
        da.longitude,
        da.latitude,
        kwargs={"polygons": polygons},
        vectorize=True,
        dask="parallelized",
    )
    return mask

In [ ]:
mask_france_wind = compute_mask_da(wind_10m_large, poly_france)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(
    mask_france_wind.values.T,
    extent=[
        mask_france_wind.longitude.min(),
        mask_france_wind.longitude.max(),
        mask_france_wind.latitude.min(),
        mask_france_wind.latitude.max(),
    ],
    origin="upper",
)

ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Mask of France used for Excat mask")

In [ ]:
# Use the mask to select the points from the dataset
wind_10m_exact = wind_10m_large.where(mask_france_wind, drop=True)
wind_10m_exact

In [ ]:
# plot the first day
fig, [ax1, ax2, ax3] = plt.subplots(
    1, 3, figsize=(15, 5), subplot_kw={"projection": ccrs.PlateCarree()}
)
extent = [min_lon, max_lon, min_lat, max_lat]
ax1.imshow(
    wind_10m_exact.isel(time=0, step=0),
    transform=ccrs.PlateCarree(),
    extent=extent,
    origin="upper",
)

ax2.imshow(
    wind_10m_large.isel(time=0, step=0),
    transform=ccrs.PlateCarree(),
    extent=extent,
    origin="upper",
)
ax3.imshow(
    da_wind_10m.isel(time=0, step=0),
    transform=ccrs.PlateCarree(),
    extent=[
        da_wind_10m.longitude.min(),
        da_wind_10m.longitude.max(),
        da_wind_10m.latitude.min(),
        da_wind_10m.latitude.max(),
    ],
    origin="upper",
)

for ax in [ax1, ax2, ax3]:
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.COASTLINE)

In [ ]:
total_wind_10m_exact = (
    wind_10m_exact.sum(dim=["latitude", "longitude"])
    .stack(time_step=("time", "step"))
    .set_index(time_step="valid_time")
)
total_wind_10m_exact

In [ ]:
df_total_wind_10m_exact = total_wind_10m_exact.to_dataframe()[["si10"]]
df_total_wind_10m_exact.plot()

In [ ]:
if save_parquet:
    df_total_wind_10m_exact.to_parquet("total_wind_10m_exact.parquet")

### Now the sun

In [ ]:
ssrd_france_exact = ssrd_france_large.where(mask_france_wind, drop=True)
ssrd_france_exact

In [ ]:
ssrd_daily_exact = ssrd_france_exact.sum(dim=["latitude", "longitude"])
ssrd_daily_exact

In [ ]:
df_ssrd_daily_exact = ssrd_daily_exact.to_dataframe()[["ssrd"]]
df_ssrd_daily_exact

In [ ]:
if save_parquet:
    df_ssrd_daily_exact.to_parquet("total_ssrd_daily_exact.parquet")

In [ ]:
df_ssrd_daily_exact.plot()

# Each regions

Now, we process each Regions 

In [ ]:
filename = "./notebooks/datascience/regions.geojson"
polys = geojson.load(open(filename))
for feature in polys["features"]:
    print(feature["properties"]["nom"])

In [ ]:
names_to_keep = [
    "Bretagne",
    "Centre-Val de Loire",
    "Grand Est",
    "Hauts-de-France",
    "Île-de-France",
    "Normandie",
    "Nouvelle-Aquitaine",
    "Occitanie",
    "Pays de la Loire",
    "Provence-Alpes-Côte d'Azur",
    "Bourgogne-Franche-Comté",
    "Auvergne-Rhône-Alpes",
    "Corse",
]

polys_region = {}
for feature in tqdm(polys["features"]):
    name = feature["properties"]["nom"]
    if name not in names_to_keep:
        continue
    if feature["geometry"]["type"] == "Polygon":
        polys_region[name] = Polygon(feature["geometry"]["coordinates"][0])
    elif feature["geometry"]["type"] == "MultiPolygon":
        # keeping the largest polygon
        tmp_list = [Polygon(geo[0]) for geo in feature["geometry"]["coordinates"]]
        largest = max(tmp_list, key=lambda x: x.area)
        polys_region[name] = Polygon(largest)


all_polys = [poly for poly in polys_region.values()]
# unpack the lists
maxi_multi_poly = MultiPolygon(all_polys)
for poly in maxi_multi_poly.geoms:
    shapely.plotting.plot_polygon(poly)

In [ ]:
masks = {}
for name, poly in polys_region.items():
    mask = xr.apply_ufunc(
        is_inside,
        wind_10m_large.longitude,
        wind_10m_large.latitude,
        kwargs={"polygons": poly},
        vectorize=True,
        dask="parallelized",
    )
    masks[name] = mask

In [ ]:
# plot all masks
fig, ax = plt.subplots()
colors = plt.cm.tab20.colors
extent = [
    mask.longitude.min(),
    mask.longitude.max(),
    mask.latitude.min(),
    mask.latitude.max(),
]
for c, mask in zip(colors, masks.values()):
    # plot the mask with the color c
    values = mask.values.T.astype(float)
    X = np.array([[c[0], c[1], c[2], alpha] for alpha in values.flatten()]).reshape(
        values.shape + (4,)
    )
    ax.imshow(X, origin="upper", cmap="viridis", extent=extent)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_title("Masks of the regions")

In [ ]:
import pickle

pickle.dump({"mask": mask_france_wind, "masks_regions": masks}, open("mask.pkl", "wb"))

In [ ]:
results_ssrd = {}
for name, mask in tqdm(masks.items()):
    ssrd = ssrd_france_large.where(mask, drop=True)
    ssrd_daily = ssrd.sum(dim=["latitude", "longitude"]).to_dataframe()["ssrd"]
    results_ssrd[name] = ssrd_daily

all_ssrd = pd.concat(results_ssrd, axis=1)
if save_parquet:
    all_ssrd.to_parquet("./notebooks/weather/all_ssrd_regions.parquet")
all_ssrd.plot()

In [ ]:
results_si10 = {}
for name, mask in tqdm(masks.items()):
    wind = wind_10m_large.where(mask, drop=True)
    wind_daily = (
        wind.sum(dim=["latitude", "longitude"])
        .stack(time_step=("time", "step"))
        .set_index(time_step="valid_time")
        .to_dataframe()["si10"]
    )
    results_si10[name] = wind_daily

all_si10 = pd.concat(results_si10, axis=1)
if save_parquet:
    all_si10.to_parquet("./notebooks/weather/all_si10_regions.parquet")
all_si10.plot()

In [ ]:
all_si10 = pd.read_parquet("./notebooks/weather/all_si10_regions.parquet")
all_ssrd = pd.read_parquet("./notebooks/weather/all_ssrd_regions.parquet")

fig, [ax1, ax2] = plt.subplots(2, 1, figsize=(15, 10), sharex=True)
all_si10.plot(ax=ax1)
all_ssrd.plot(ax=ax2)

# Processing all available forcast horizons

In order to asses the performance of the forecast prediction, we will extract all the available forcasts.


In [ ]:
from pathlib import Path
import pandas as pd
import xarray as xr
import numpy as np
from tqdm.auto import tqdm
import matplotlib.pyplot as plt

from shapely.geometry import Point, Polygon, MultiPolygon
import shapely.plotting
import geojson
from dask.diagnostics import ProgressBar

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pickle


pbar = ProgressBar()
pbar.register()

In [ ]:
metropole_geojson_file = "./notebooks/datascience/metropole.geojson"
metropole = geojson.load(open(metropole_geojson_file))
polys_france = [Polygon(geo[0]) for geo in metropole["geometry"]["coordinates"]]
areas = [p.area for p in polys_france]

min_area = 0.2  # threshold to remove small islands
polys_france = [p for p, area in zip(polys_france, areas) if area > min_area]
print(f"number of polygons: {len(polys_france)}")
poly_france = MultiPolygon(polys_france)
# poly_france = polys_france[0]  # france metropolitan

bouns = poly_france.envelope.bounds
min_lon = bouns[0]
max_lon = bouns[2]
min_lat = bouns[1]
max_lat = bouns[3]

In [ ]:
the_masks = pickle.load(open("mask.pkl", "rb"))
mask_france_wind = the_masks["mask"]
masks_regions = the_masks["masks_regions"]

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
# apply formatter to the logger
logger.handlers[0].setFormatter(formatter)


logging.info("This is an info message")

In [ ]:
start_date = "2022-02-01"
end_date = "2024-04-08"
list_date = pd.date_range(start=start_date, end=end_date, freq="D")
list_date = list_date.strftime("%Y-%m-%d")
cases_forecast = {
    "J+0": ["00H12H", "13H24H"],
    "J+1": ["25H36H", "37H48H"],
    "J+2": ["49H60H", "61H72H"],
    "J+3": ["73H84H", "85H96H"],
}
cases = [
    "00H12H",
    "13H24H",
]  # "25H36H", "37H48H", "49H60H", "61H72H", "73H84H", "85H96H", "97H102H"]
filename_to_save_template = (
    "/shared/home/antoine-2etavant/data/arpege/{date}_SP1_{case}.grib2"
)

KEYS_FILTER_SSPD = {
    "typeOfLevel": "surface",
    "cfVarName": "ssrd",
}
KEYS_FILTER_WIND = {
    "typeOfLevel": "heightAboveGround",
    "level": 10,
    "cfVarName": "si10",
}
KEYS_FILTER_T2M = {
    "typeOfLevel": "heightAboveGround",
    "level": 2,
    "cfVarName": "t2m",
}

In [ ]:
def list_of_filenames_from_case(horizon: str):
    morning_case, afternoon_case = cases_forecast[horizon]
    filenames_afternoon = [
        filename_to_save_template.format(date=date, case=afternoon_case)
        for date in list_date
    ]
    filenames_afternoon_exist = [
        filename for filename in filenames_afternoon if Path(filename).exists()
    ]
    filenames_afternoon = filenames_afternoon_exist
    filenames_morning = [
        filename_to_save_template.format(date=date, case=morning_case)
        for date in list_date
    ]
    filenames_morning_exist = [
        filename for filename in filenames_morning if Path(filename).exists()
    ]
    filenames_morning = filenames_morning_exist
    return filenames_morning, filenames_afternoon

In [ ]:
def filter_filenames(filenames, KEYS_FILTER):
    return [
        filename
        for filename in filenames
        if xr.open_dataset(
            filename, engine="cfgrib", backend_kwargs={"filter_by_keys": KEYS_FILTER}
        ).data_vars
    ]

In [ ]:
def drop_no_step(ds):
    # Check if 'step' is in the dataset's coordinates
    # Used to find files that are not working properly
    if "step" not in ds.coords:
        display(ds)
    return ds

In [ ]:
def process_ssrd(
    horizon: str,
    mask_france=mask_france_wind,
    masks_regions=masks_regions,
    bounds_france=[min_lon, max_lon, min_lat, max_lat],
    save_parquet=False,
    process_regions=False,
):
    _, filenames_afternoon = list_of_filenames_from_case(horizon)
    filenames_afternoon = filter_filenames(filenames_afternoon, KEYS_FILTER_SSPD)

    logging.info(f"Processing {horizon} with {len(filenames_afternoon)} files")
    da_ssrd: xr.DataArray = xr.open_mfdataset(
        filenames_afternoon,
        engine="cfgrib",
        parallel=True,
        backend_kwargs={"filter_by_keys": KEYS_FILTER_SSPD},
        concat_dim="time",
        combine="nested",
        preprocess=drop_no_step,
    ).ssrd
    min_lon, max_lon, min_lat, max_lat = bounds_france
    if horizon == "J+0":
        step = np.timedelta64(1, "D")
    elif horizon == "J+1":
        step = np.timedelta64(2, "D")
    elif horizon == "J+2":
        step = np.timedelta64(3, "D")
    elif horizon == "J+3":
        step = np.timedelta64(4, "D")
    ssrd_france_large = da_ssrd.sel(step=step).where(
        (da_ssrd.longitude > min_lon)
        & (da_ssrd.longitude < max_lon)
        & (da_ssrd.latitude > min_lat)
        & (da_ssrd.latitude < max_lat),
        drop=True,
    )
    total_sun_flux_large = ssrd_france_large.sum(dim=["latitude", "longitude"])
    logging.info(f"Total sun flux large: {total_sun_flux_large}")
    ssrd_france_exact = ssrd_france_large.where(mask_france, drop=True)
    total_sun_flux_exact = ssrd_france_exact.sum(dim=["latitude", "longitude"])
    logging.info(f"Total sun flux exact: {total_sun_flux_exact}")

    if process_regions:
        results_ssrd = {}
        for name, mask in tqdm(masks_regions.items(), leave=False):
            ssrd = ssrd_france_large.where(mask, drop=True)
            ssrd_daily = ssrd.sum(dim=["latitude", "longitude"]).to_dataframe()["ssrd"]
            results_ssrd[name] = ssrd_daily

        all_ssrd = pd.concat(results_ssrd, axis=1)

    if save_parquet:
        prefix = (
            "./notebooks/weather/forcasts/"
            + horizon
            + "_"
            + str(start_date)
            + "_"
            + str(end_date)
            + "_"
        )
        if process_regions:
            logging.info(f"Saving to {prefix}")
            all_ssrd.to_parquet(prefix + "sun_flux_regions.parquet")
            logging.info("Saved to regions")
        else:
            # total_sun_flux_large.to_dataframe()[["ssrd"]].to_parquet(
            #     prefix + "sun_flux_large.parquet"
            # )
            # logging.info("Saved to large")
            total_sun_flux_exact.to_dataframe()[["ssrd"]].to_parquet(
                prefix + "sun_flux_exact.parquet"
            )
            logging.info("Saved to exact")

In [ ]:
def process_wind(
    horizon: str,
    mask_france=mask_france_wind,
    masks_regions=masks_regions,
    bounds_france=[min_lon, max_lon, min_lat, max_lat],
    save_parquet=False,
    process_regions=False,
):
    filenames_morning, filenames_afternoon = list_of_filenames_from_case(horizon)
    filenames_morning = filter_filenames(filenames_morning, KEYS_FILTER_WIND)
    filenames_afternoon = filter_filenames(filenames_afternoon, KEYS_FILTER_WIND)

    logging.info(f"Processing {horizon} with {len(filenames_morning)} files")
    da_wind_10m_morning = xr.open_mfdataset(
        filenames_morning,
        engine="cfgrib",
        parallel=True,
        backend_kwargs={"filter_by_keys": KEYS_FILTER_WIND},
        concat_dim="time",
        combine="nested",
    ).si10
    da_wind_10m_afternoon = xr.open_mfdataset(
        filenames_afternoon,
        engine="cfgrib",
        parallel=True,
        backend_kwargs={"filter_by_keys": KEYS_FILTER_WIND},
        concat_dim="time",
        combine="nested",
    ).si10
    da_wind_10m = xr.concat(
        [da_wind_10m_morning, da_wind_10m_afternoon],
        dim="step",
    )
    min_lon, max_lon, min_lat, max_lat = bounds_france
    wind_10m_large = da_wind_10m.where(
        (da_wind_10m.longitude > min_lon)
        & (da_wind_10m.longitude < max_lon)
        & (da_wind_10m.latitude > min_lat)
        & (da_wind_10m.latitude < max_lat),
        drop=True,
    )
    total_wind_10m_large = (
        wind_10m_large.sum(dim=["latitude", "longitude"])
        .stack(time_step=("time", "step"))
        .set_index(time_step="valid_time")
    )
    logging.info(f"Total wind 10m large: {total_wind_10m_large}")

    wind_10m_exact = wind_10m_large.where(mask_france, drop=True)
    total_wind_10m_exact = (
        wind_10m_exact.sum(dim=["latitude", "longitude"])
        .stack(time_step=("time", "step"))
        .set_index(time_step="valid_time")
    )
    logging.info(f"Total wind 10m exact: {total_wind_10m_exact}")

    if process_regions:
        results_si10 = {}
        for name, mask in tqdm(masks_regions.items(), leave=False):
            wind = wind_10m_large.where(mask, drop=True)
            wind_daily = (
                wind.sum(dim=["latitude", "longitude"])
                .stack(time_step=("time", "step"))
                .set_index(time_step="valid_time")
                .to_dataframe()["si10"]
            )
            results_si10[name] = wind_daily

        all_si10 = pd.concat(results_si10, axis=1)

    if save_parquet:
        prefix = (
            "./notebooks/weather/forcasts/"
            + horizon
            + "_"
            + str(start_date)
            + "_"
            + str(end_date)
            + "_"
        )
        logging.info(f"Saving to {prefix}")
        if process_regions:
            all_si10.to_parquet(prefix + "wind_regions.parquet")
            logging.info("Saved to regions")
        else:
            # total_wind_10m_large.to_dataframe()[["si10"]].to_parquet(
            #     prefix + "wind_large.parquet"
            # )
            # logging.info("Saved to large")
            total_wind_10m_exact.to_dataframe()[["si10"]].to_parquet(
                prefix + "wind_exact.parquet"
            )
            logging.info("Saved to exact")

In [ ]:
try:
    pbar.unregister()
except KeyError:
    pass

In [ ]:
for horizon in ["J+1", "J+2", "J+3"]:
    start_date = "2022-02-01"
    end_date = "2023-01-15"
    list_date = pd.date_range(start=start_date, end=end_date, freq="D")
    list_date = list_date.strftime("%Y-%m-%d")
    logging.info(f"Processing {horizon} {start_date} to {end_date}")
    process_ssrd(horizon, save_parquet=True)
    process_wind(horizon, save_parquet=True)

In [ ]:
for horizon in ["J+0", "J+1", "J+2", "J+3"]:
    start_date = "2023-01-15"
    end_date = "2024-04-08"
    list_date = pd.date_range(start=start_date, end=end_date, freq="D")
    list_date = list_date.strftime("%Y-%m-%d")
    logging.info(f"Processing {horizon} {start_date} to {end_date}")
    process_ssrd(horizon, save_parquet=True)
    process_wind(horizon, save_parquet=True)

# concat the forecasts to predictions

In [ ]:
datas = {}
for horizon in ["J+0", "J+1", "J+2", "J+3"]:
    start_date = "2022-02-01"
    end_date = "2023-01-15"
    prefix = (
        "./notebooks/weather/forcasts/"
        + horizon
        + "_"
        + str(start_date)
        + "_"
        + str(end_date)
        + "_"
    )
    sun_exact_filename = prefix + "sun_flux_exact.parquet"
    wind_exact_filename = prefix + "wind_exact.parquet"
    datas[horizon + "_sun_exact"] = pd.read_parquet(sun_exact_filename)
    datas[horizon + "_wind_exact"] = pd.read_parquet(wind_exact_filename)

    start_date = "2023-01-15"
    end_date = "2024-04-08"
    prefix = (
        "./notebooks/weather/forcasts/"
        + horizon
        + "_"
        + str(start_date)
        + "_"
        + str(end_date)
        + "_"
    )
    sun_exact_filename = prefix + "sun_flux_exact.parquet"
    wind_exact_filename = prefix + "wind_exact.parquet"
    datas[horizon + "_sun_exact"] = pd.concat(
        [
            datas[horizon + "_sun_exact"],
            pd.read_parquet(sun_exact_filename).loc["2023-01-16":],
        ],
        axis=0,
    )
    try:
        df2_wind = pd.read_parquet(wind_exact_filename).loc["2023-01-16":]
    except KeyError:
        df2_wind = pd.read_parquet(wind_exact_filename)
    datas[horizon + "_wind_exact"] = pd.concat(
        [datas[horizon + "_wind_exact"], df2_wind[~df2_wind.index.duplicated()]], axis=0
    )

    datas[horizon + "_sun_exact"].rename(
        columns={"ssrd": horizon + "_sun_exact"}, inplace=True
    )
    datas[horizon + "_wind_exact"].rename(
        columns={"si10": horizon + "_wind_exact"}, inplace=True
    )
    datas[horizon + "_wind_exact"] = datas[horizon + "_wind_exact"].resample("D").mean()

In [ ]:
datas.keys()

In [ ]:
mean_forecasts = pd.concat(datas.values(), axis=1)

In [ ]:
# The Sun flux is accumulated from the very beginning of the forecast, not the day !

In [ ]:
mean_forecasts["J+3_sun_exact"] -= mean_forecasts["J+2_sun_exact"]
mean_forecasts["J+2_sun_exact"] -= mean_forecasts["J+1_sun_exact"]
mean_forecasts["J+1_sun_exact"] -= mean_forecasts["J+0_sun_exact"]

In [ ]:
mean_forecasts.to_csv("./notebooks/weather/mean_forecasts.csv")

In [ ]:
mean_forecasts.filter(like="sun").plot()

In [ ]:
mean_forecasts.filter(like="wind").plot()